In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook as tqdm
import feather
import pandas as pd
import numpy as np

In [2]:
URL = 'http://www.geocoding.jp/api/'


def coordinate(address):
    """
    addressに住所を指定すると緯度経度を返す。

    >>> coordinate('東京都文京区本郷7-3-1')
    ['35.712056', '139.762775']
    """
    payload = {'q': address}
    html = requests.get(URL, params=payload)
    soup = BeautifulSoup(html.content, "html.parser")
    if soup.find('error'):
        try:
            address = address.split("(")[0] + address.split(")")[1]
            payload = {'q': address}
            html = requests.get(URL, params=payload)
            soup = BeautifulSoup(html.content, "html.parser")
            if soup.find('error'):
                latitude = "-999.0"
                longitude = "-999.0"
            else:
                latitude = soup.find('lat').string
                longitude = soup.find('lng').string
        
        except:
            latitude = "-999.0"
            longitude = "-999.0"
        #raise ValueError(f"Invalid address submitted. {address}")
    else:
        latitude = soup.find('lat').string
        longitude = soup.find('lng').string
    return (latitude, longitude)


def coordinates(addresses, interval=5, progress=True):
    """
    addressesに住所リストを指定すると、緯度経度リストを返す。

    >>> coordinates(['東京都文京区本郷7-3-1', '東京都文京区湯島３丁目３０−１'], progress=False)
    [['35.712056', '139.762775'], ['35.707771', '139.768205']]
    """
    coordinates = {}
    for address in progress and tqdm(addresses) or addresses:
        coordinates[address] = (coordinate(address))
        time.sleep(interval)
    return coordinates

In [3]:
ido_kedo = feather.read_dataframe("../input/eki_ido_kedo.feather")

In [4]:
"""%%time
idx = ido_kedo["駅緯度"] == -999.0
nokori_dict = coordinates(list(ido_kedo["駅名"][idx].values), progress=False)"""

'%%time\nidx = ido_kedo["駅緯度"] == -999.0\nnokori_dict = coordinates(list(ido_kedo["駅名"][idx].values), progress=False)'

In [5]:
"""feather output"""
"""
temp = pd.DataFrame(nokori_dict).T
temp = temp.reset_index()
temp.columns = ["駅名", "駅緯度", "駅経度"]
temp.to_feather("../input/eki_ido_kedo_nokori.feather")"""

'\ntemp = pd.DataFrame(nokori_dict).T\ntemp = temp.reset_index()\ntemp.columns = ["駅名", "駅緯度", "駅経度"]\ntemp.to_feather("../input/eki_ido_kedo_nokori.feather")'

In [6]:
train = pd.read_csv('../input/train.csv')
target = train["賃料"]
train = train.drop("賃料", axis=1)
test = pd.read_csv('../input/test.csv')

train_length = train.shape[0]
test_length = test.shape[0]
all_df = pd.concat([train, test], axis=0, ignore_index=True)

In [7]:

len(list(all_df["所在地"].unique()))
#list(all_df["所在地"].unique())
#list(all_df["所在地"].unique())[5:6]

17199

In [8]:
%%time
n_split = 100
module = len(list(all_df["所在地"].unique()))//n_split
for i in tqdm(range(95,n_split)):
    address_dict = coordinates(list(all_df["所在地"].unique()[i*module:(i+1)*module]), progress=False)
    temp = pd.DataFrame(address_dict).T
    temp = temp.reset_index()
    temp.columns = ["住所", "住所緯度", "住所経度"]
    temp.to_feather(f"../input/address_ido_kedo_{i}.feather")


CPU times: user 7.53 s, sys: 821 ms, total: 8.35 s
Wall time: 1h 26min 6s


In [78]:
"""feather output"""
temp = pd.DataFrame(address_dict).T
temp = temp.reset_index()
temp.columns = ["住所", "住所緯度", "住所経度"]
temp.to_feather(f"../input/address_ido_kedo.feather_{i}")

NameError: name 'address_dict' is not defined

In [11]:
#feather.read_dataframe("../input/address_ido_kedo_1.feather")

,住所,住所緯度,住所経度
0,東京都中央区日本橋横山町,35.693475,139.78406
1,東京都板橋区仲宿,35.755663,139.709646
2,東京都足立区伊興２丁目,35.792416,139.781184
3,東京都墨田区押上１丁目,-999.0,-999.0
4,東京都豊島区東池袋４丁目,35.727032,139.721138
5,東京都台東区根岸５丁目12-13,35.726101,139.78705
6,東京都墨田区東向島４丁目24-8,35.724771,139.817644
7,東京都墨田区文花１丁目,35.708995,139.822101
8,東京都練馬区春日町６丁目16-11,35.753826,139.631705
9,東京都江戸川区船堀７丁目,35.677387,139.869446
